In [ ]:
#Bibliotheken laden
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly_express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [ ]:
# Datensatz einlesen
df_funds = pd.read_pickle('funds_after_preprocessing.pkl')
df_funds.head()

# Finanzierungsstatus aller Projekte

In [ ]:
# Pie Chart der Erfolgsraten
df_status = df_funds.groupby(['funding_status'], as_index=False).agg(anzahl=('funding_status', 'size'),
                                                                     Zielbetrag_gesamt = ('loan_amount', 'sum'),
                                                                     ausgezahlt_gesamt = ('funded_amount', 'sum'),
                                                                     Differenz = ('difference_amount', 'sum')
                                                                    )
df_status

In [ ]:
print(sum(df_status['ausgezahlt_gesamt'])) # Summe der ausgezahlten Beträge
print(sum(df_status['Differenz'])) # Summe der Differenzbeträge
print(sum(df_status['Differenz'])/sum(df_status['ausgezahlt_gesamt']))

In [ ]:
fig = px.pie(df_status, 
             values='anzahl', 
             names='funding_status',
             color='funding_status',
             color_discrete_map={'unfunded':'red',
                                 'partially funded':'orange',
                                 'funded':'green',
                                 'overfunded':'purple'},
             title='Bewertung: Übersicht Finanzierungsstatus aller Projekte'
            )
fig.show()

Die Grafik veranschaulicht: erfolgreich finanzierte Projekte haben einen Anteil  von 92,8%, andere 7,2%. 

Differenzbeträge zeigen, dass **insgesamt knapp 38 Millionen \\$ nicht investiert wurden** und ist auch vielleicht ein Hinweis darauf, **wieviel Provision verloren ging**. Auch wenn es keine Angabe zur Provision gibt, wird höchstwahrscheinlich eine höhere Provision fällig bei erfolgreicher Finanzierung, die auch noch gestaffelt ist je nach Finanzierungsbetrag. Ähnlich wie bei Ebay wird eine Verkaufsprovision erhoben, wenn der Artikel verkauft wurde und keine, wenn der Artikel nicht verkauft wurde.

# Einblick in teilweise oder nichtfinanzierte Projekte
Im Folgenden werden die nicht erfolgreich finanzierten Projekte näher angeschaut. Zunächst werden Kategorien vergeben, die eine Spanne des angeforderten Geldbetrags darstellen.  
Die **Betragskategorien** sind wie folgt:
- <1000\$, 
- 1000...9999\$
- 10k...19999\$
- 20k...99999$
- 100k\$

In [ ]:
# Erstellung der Betragskategorien
kategorien = ['<1000', '1000...9999', '10k...19999', '20k...99999', '100k']

einteilung = [0,999,9999,19999,99999,np.inf] # np.inf als max
# ein Wert mehr als Kategorien: 0

df_funds['Betragskategorie'] = pd.cut(df_funds['loan_amount'], bins=einteilung, labels=kategorien)
df_funds['Betragskategorie'].unique()

In [ ]:
# Check, ob das passt
df_funds.loc[(df_funds['success_rate']<100) & (df_funds['success_rate']>80), ['success_rate']]

Des Weiteren wird eine Erfolgskategorie eingeführt, um die Erfolgsquote gruppieren zu können:
Die **Erfolgskategorien** sind wie folgt:
- <1%, 
- 1...<20%
- 20...<40
- 40...<60%
- 60...<80%
- 80...<100%
- 100%

In [ ]:
# Erstellung der Erfolgskategorien
kategorien = ['<1', '1...<20', '20...<40', '40...<60', '60...<80', '80...<100', '100']

einteilung = [-1,0.999999,19.999999,39.999999,59.999999,79.999999, 99.999999, np.inf] # np.inf als max
# ein Wert mehr als Kategorien: 0

df_funds['Erfolgskategorie'] = pd.cut(df_funds['success_rate'], bins=einteilung, labels=kategorien)
df_funds['Erfolgskategorie'].unique()


In [ ]:
# Erstelle Spalte mit 1 für mindestens 100% Efolg, 0 für weniger als 100% Erfolgsrate
l_erfolgreich = []
for i in df_funds['success_rate']:
    if i >= 100:
        l_erfolgreich.append(1)
    else:
        l_erfolgreich.append(0)
df_funds['erfolgreich'] = l_erfolgreich

In [ ]:
# Gruppierte Projekte nach Betragskategorie, wieviel gesamt, wieviel erfolgrich
df_betrag = df_funds.groupby('Betragskategorie', as_index=False).agg(Gesamt=('Betragskategorie', 'size'),
                                                                     Erfolg=('erfolgreich', 'sum'),
                                                                     Zielbetrag_gesamt=('loan_amount', 'sum'),
                                                                     Ausgezahlt_gesamt=('funded_amount', 'sum'),
                                                                     Differenzbetrag = ('difference_amount', 'sum')
                                                                    )
# df_betrag['Differenz_Anteil'] = df_betrag['Differenzbetrag']/df_betrag['Zielbetrag_gesamt']*100
df_betrag['Misserfolg'] = df_betrag['Gesamt'] - df_betrag['Erfolg']

df_betrag

Obige Tabelle zeigt alle Beträge gestaffelt nach Betragskategorie.  
Die weiteren Analysen werden sich auf die Differenzbeträge beziehen.

In [ ]:
# Check, ob das passt erfolgreiche Projekte pro Betragskategorie
for i in kategorien:
    print(len(df_funds.loc[(df_funds['Betragskategorie']==i) & (df_funds['success_rate'] >=100), 
             ['Betragskategorie', 'success_rate']
            ])
         )

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
df_temp = df_betrag.drop(4)

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Bar(name='Differenzbeträge in $',
                     x=df_temp['Betragskategorie'],
                     y=df_temp['Differenzbetrag'],
                     text = df_temp['Differenzbetrag']
                     ),secondary_y=False,)

fig.add_trace(go.Bar(name='Zielbeträge in $',
                     x=df_temp['Betragskategorie'],
                     y=df_temp['Zielbetrag_gesamt'],
                     text = df_temp['Zielbetrag_gesamt']
                     ),secondary_y=False)

# Change the bar mode
fig.update_layout(barmode='group',
                  title='Bewertung: Summierte Differenzbeträge und Zielbeträge je Finanzierungswert',
                  title_font_size = 20,
                  width = 1000,
                  height = 800,
                  legend_xanchor = 'right',
                  legend_x = 0.9
                 )
fig.update_yaxes(type="log", title_text='Beträge in $', title_font_size = 18)
fig.update_xaxes(title_text='Finanzierungswert der einzelnen Projekte in $', title_font_size = 18)
#fig.update_traces(marker_opacity=0.5, selector=dict(type='bar')) 
fig.show()

In [ ]:
# Korrelation scatter Anstieg loan_amount, Anstieg Differenzen

df = df_betrag.sort_values('Zielbetrag_gesamt')
fig = px.scatter(df, x='Zielbetrag_gesamt', y='Differenzbetrag', color = 'Betragskategorie',symbol='Betragskategorie')

fig.update_layout(title='Differenzbeträge zu Zielbeträge',
                  title_font_size = 20,
                  width = 600,
                  height = 600,
                  legend_xanchor = 'left',
                  legend_x = 0.3
                 )
fig.update_yaxes(title_text='Differenzeträge in $', title_font_size = 18)
fig.update_xaxes(title_text='Zielbeträge in $', title_font_size = 18)
fig.show()

Der obige Barchart hat eine logarithmierte Y-Achse, um die kleinen Beträge mit abbilden zu können.  
Es ist ein Zusammenhang zu erkennen zwischen der Differenz und den Zielbeträgen. Je höher die Summe der Zielbeträge, desto höher die Summe der Differenzbeträge.  
Er verdeutlicht, dass die meisten Differenzen bei Projekten von 1000\\$ bis 9999\\$ bestehen. Es sind aber auch gleichzeitig die, bei denen am meisten Geld angefragt wurde. Es muss dort noch etwas etwas geben, was die Investoren scheut, die Projekte vollständig zu finanzieren. Es lohnt sich daher tiefer zu graben in der Kategorie, die die höchsten Beträge generiert, aber gleichzeitig auch am schlechtesten finanziert werden. Die Korrelation zwischen Anstieg der Zielsumme und Differenzbeträgen lässt sich wahrscheinlich auf andere Kategorien übertragen. 

# Aufschlüsselung der Ursachen nicht investierter Beträge anhand der Kategorie 1000 bis 9999\\$
Es folgen neue Tabellen, die die Abhängigkeit der Differenzbeträge von verschiedenen Faktoren beschreiben.

In [ ]:
# summierte Differenzbeträge abhängig vom Repayment Intervall
df_1000 = df_funds.loc[df_funds['Betragskategorie'].isin(['1000...9999']),:]
# df_1000['sector'] = df_1000['sector'].cat.remove_unused_categories()
df_1000_rep = df_1000.groupby(['repayment_interval'], observed = True, as_index=False).agg(Differenzbetrag=('difference_amount','sum'),                                                  
                                                 )
df_1000_rep.sort_values('Differenzbetrag', inplace=True)

df_1000_rep

In [ ]:
# summierte Differenzbeträge abhängig vom Sector
df_1000_sector = df_1000.groupby(['sector'], observed = False, as_index=False)\
.agg(Differenzbetrag=('difference_amount','sum'))
df_1000_sector.sort_values('Differenzbetrag', inplace=True)
df_1000_sector['sector'] = df_1000_sector['sector'].cat.remove_unused_categories()
df_1000_sector

In [ ]:
# summierte Differenzbeträge abhängig vom Land
df_1000_country = df_1000.groupby(['country', 'country_code'], observed = True, as_index=False)\
.agg(Differenzbetrag=('difference_amount','sum'))
df_1000_country.drop(df_1000_country.loc[df_1000_country['Differenzbetrag']==0,:].index, inplace=True)
df_1000_country.sort_values('Differenzbetrag', inplace=True)
df_1000_country

In [ ]:
# summierte Differenzbeträge abhängig von der Laufzeit
df_1000_term = df_1000.groupby(['term_in_months'], observed = True, as_index=False)\
.agg(Differenzbetrag=('difference_amount','sum'))
df_1000_term.sort_values('term_in_months', inplace=True)
df_1000_term.drop(df_1000_term.loc[df_1000_term['Differenzbetrag']==0,:].index, inplace=True)
df_1000_term

In [ ]:
# summierte Differenzbeträge abhängig vom Erfolg
df_1000_success = df_1000.groupby(['Erfolgskategorie'], observed = True, as_index=False)\
.agg(Differenzbetrag=('difference_amount','sum'))
# df_1000_success.sort_values('Differenzbetrag', inplace=True)
# df_1000_success.drop(df_1000_term.loc[df_1000_term['Differenzbetrag']==0,:].index, inplace=True)
df_1000_success

In [ ]:
# Übersicht der obigen gruppierten Dataframes
fig = make_subplots(rows=5, cols=1)
fig.add_trace(go.Bar(name='Differenz nach Rückzahlungsintervall',
                     x=df_1000_rep['repayment_interval'],
                     y=df_1000_rep['Differenzbetrag'],
                     text = df_1000_rep['Differenzbetrag'],
                     ), row = 1, col= 1)
fig.add_trace(go.Bar(name='Differenz nach Sektor',
                     x=df_1000_sector['sector'],
                     y=df_1000_sector['Differenzbetrag'],
                     text = df_1000_sector['Differenzbetrag'],
                     ), row = 2, col= 1)
fig.add_trace(go.Scatter(name='Differenz nach Laufzeit',
                     x=df_1000_term['term_in_months'],
                     y=df_1000_term['Differenzbetrag'],
                     text = df_1000_term['Differenzbetrag'],
                         mode='markers'
                     ), row = 3, col= 1)
fig.add_trace(go.Bar(name='Differenz nach Land',
                     x=df_1000_country['country_code'],
                     y=df_1000_country['Differenzbetrag'],
                     text = df_1000_country['Differenzbetrag'],
                     ), row = 4, col= 1)
fig.add_trace(go.Scatter(name='Differenz nach Erfolgskategorie',
                     x=df_1000_success['Erfolgskategorie'],
                     y=df_1000_success['Differenzbetrag'],
                     text = df_1000_term['Differenzbetrag'],
                     ), row = 5, col= 1)

fig.update_yaxes(title_text='Differenzeträge in $', title_font_size = 12)
fig.update_xaxes(title_text='Laufzeit in Monaten', row=3, col=1)
fig.update_xaxes(title_text='Ländercodes', row=4, col=1)
fig.update_xaxes(title_text='Erfolg in %', row=5, col=1)
fig.update_layout(title='Bewertung: Differenzbeträge in verschiedenen Kategorien',
                  title_font_size = 20,
                  width = 1200,
                  height = 1600,
                  legend_xanchor = 'left',
                  legend_x = 0.1,
                 )
fig.show()

**Zusammenfassung der obigen Plots:**
- Plot 1 Rückzahlungsintervall: monatliche Rückzahlung erzeugt für sich die höchsten Differenzbeträge: 58% aller Differenzbeträge
- Plot 2 Sector: Agriculture für sich erzeugt die höchsten Differenzbeträge: 25% aller Differenzbeträge
- Plot 3 Laufzeit: irgendwo zwischen 10 und 30 Monaten Laufzeit sind die höchsten Differenzbeträge
- Plot 4 Länder: in die USA wurde am wenigsten investiert: 21% aller Differenzbeträge
- Plot 5 Erfolgskategorie: Anstieg er Differenzbeträge bis 1...<20 %, danach sinken sie

Die Differenzbeträge in der Kategorie mit einen Zielbetrag zwischen 1000 bis 9999\\$ hängen von verschiedenen Ursachen ab.


In [ ]:
df_1000_monthly = df_1000.loc[(df_1000['repayment_interval']=='monthly') & (df_1000['Erfolgskategorie'].isin(['1...<20'])),:]
df_1000_monthly.head(1)

In [ ]:
df_1000_group = df_1000_monthly.groupby(['sector','country'], observed = True, as_index=False)\
.agg(Differenzbetrag=('difference_amount','sum'),
     Zielbetrag = ('loan_amount', 'sum')
    )
df_1000_group.drop(df_1000_group.loc[df_1000_group['Differenzbetrag']==0,:].index, inplace=True)
df_1000_group.sort_values('Differenzbetrag', inplace=True)
df_1000_group

In [ ]:
fig = px.scatter(df_1000_group, x='Erfolgskategorie', y='difference_amount',
                 color='sector',
                 size='difference_amount'
                )
fig.show()

In [ ]:
# Scatterplot des Entertainment-subsets
fig = px.scatter(df_entertainment,
x='loan_amount',
y='funding_percent',
color='country',
size='borrower_number',
template='seaborn',
title='funding percentage per loan size and country in the Entertainment sector',
hover_data=["country", "loan_amount",'funding_percent','use']
)
# hinzufuegen der Infobox ueber die Bubble size
fig.add_annotation(text='Size of the Bubble: Number of People requesting the loan', 
align='right',
showarrow=False,
xref='paper',
yref='paper',
x=1.0,
y=-0.15,
bordercolor='grey',
borderwidth=1)

fig.show()

In [ ]:
sum(df_1000_group['Differenzbetrag'])